In [124]:
import sys
print(sys.executable)

import mintapi
import imaplib
import email
import time
import datetime
import pandas as pd

email_user = '544wallerhoa@gmail.com'
email_pass = 'hyjjyr-wYjwoc-3jimti'
mint_pwd = "toxwi4-xopgic-cucsuB"
SMTP_SERVER = "imap.gmail.com"
SMTP_PORT = 993


/Users/bnlerner/anaconda3/envs/hoa/bin/python


In [73]:
def read_email_from_gmail():

    time.sleep(5)

    mail = imaplib.IMAP4_SSL(SMTP_SERVER, SMTP_PORT)
    mail.login(email_user, email_pass)
    mail.select()

    typ, message_numbers = mail.search(None, 'ALL')
    
    num = message_numbers[0].split()[-1]
    typ, msg = mail.fetch(num, '(RFC822)')
    msg = email.message_from_bytes(msg[0][1])
    
    mail.close()
    mail.logout()
    
    for part in msg.walk():
        
        def find_num(to_find_msg):
            for word in to_find_msg:
                try:
                    num = int(word)
                    if len(word) == 6:
                        return num
                except:
                    pass

            raise RuntimeError('did not find the mfa')
            
        msg_type = part.get_content_type()
        if "multipart" in msg_type:
            continue
        elif "text/html" in msg_type:
            final_msg = part.get_payload(decode=1).decode().replace('\n','').replace(' ', '').split('\r')
            return find_num(final_msg)
        else:
            final_msg = part.get_payload(decode=1).decode().split()
            return find_num(final_msg)
        
    


In [76]:
def get_mint_info():

    # A callback accepting a single argument (the prompt)
    # which returns the user-inputted 2FA code. By default
    # the default Python `input` function is used.
    mint = mintapi.Mint(email_user, mint_pwd, mfa_method='email',
                        headless=True, mfa_input_callback=read_email_from_gmail)

    # Initiate an account refresh
    mint.initiate_account_refresh()

    # Get basic account information
    #accounts = mint.get_accounts()
    nw = mint.get_net_worth()

    hoa_account = 'BUSINESS CHECKING'

    # Get transactions
    df = mint.get_transactions(include_investment=False) # as pandas dataframe
    
    return df[df['account_name'] == hoa_account], nw


In [121]:
#df, net_worth = get_mint_info()
df.sort_values(by=['transaction_type', 'amount', 'category'], ascending=[True, False, False]).reset_index(inplace=True)
df = df[['date', 'description', 'original_description', 'amount', 'transaction_type', 'category']]
df

,date,description,original_description,amount,transaction_type,category
0,2018-11-05,Kleban R Custom,FROM KLEBAN R CUSTOM MANAGEMENT(RECURRING TRAN...,450.00,credit,hoa payment
1,2018-10-31,Deposit Bank By,DEPOSIT FROM BANK BY MAIL,450.00,credit,hoa payment
2,2018-10-05,Kleban R Custom,FROM KLEBAN R CUSTOM MANAGEMENT(RECURRING TRAN...,450.00,credit,hoa payment
3,2018-10-03,Deposit Bank By,DEPOSIT FROM BANK BY MAIL,450.00,credit,hoa payment
4,2018-09-05,Kleban R Custom,FROM KLEBAN R CUSTOM MANAGEMENT(RECURRING TRAN...,450.00,credit,hoa payment
5,2018-08-27,Deposit Bank By,DEPOSIT FROM BANK BY MAIL,450.00,credit,hoa payment
6,2018-11-05,Edeposit In Branch,eDeposit in Branch/Store11/03/18 12:00:51 PM 8...,230.00,credit,hoa payment
7,2018-10-09,Edeposit In Branch,eDeposit in Branch/Store10/09/18 05:44:53 PM 1...,230.00,credit,hoa payment
8,2018-09-07,Edeposit In Branch 2100 Fillmore Street,eDeposit in Branch/Store09/07/18 01:43:13 PM 2...,230.00,credit,hoa payment
9,2018-09-28,Edeposit In Branch 557 Castro Street,eDeposit in Branch/Store09/28/18 03:53:50 PM 5...,9.27,credit,income


In [95]:
today = datetime.datetime.now()
one_month = today.replace(month=today.month-1)

In [96]:
df_cash_flow = df[df.date >= one_month].groupby(['transaction_type']).sum()

In [109]:
tot_cash_flow = df_cash_flow.loc['credit', :] - df_cash_flow.loc['debit', :]

In [112]:
df_cat = df.groupby(['category', 'transaction_type']).sum()

In [190]:
df_i = pd.Series(name=('Total', 'Total'))
df_i['amount'] = tot_cash_flow.values[0]
df_cat = df_cat.append(df_i)
df_cat.reset_index(inplace=True)

In [196]:
df_cat.to_html('test_html.html')
#df_cat.iloc[1,:]

In [46]:
mail = imaplib.IMAP4_SSL(SMTP_SERVER, SMTP_PORT)
mail.login(email_user, email_pass)
mail.select()

typ, message_numbers = mail.search(None, 'ALL')

num = message_numbers[0].split()[-1]
typ, msg = mail.fetch(num, '(RFC822)')
msg = email.message_from_bytes(msg[0][1])

for part in msg.walk():

    msg_type = part.get_content_type()
    #if "multipart" in msg_type or "text/html" in msg_type:
    #    continue

    final_msg = part.get_payload(decode=1).decode().split()


mail.close()
mail.logout()

('BYE', [b'LOGOUT Requested'])

In [197]:
import mimetypes

In [198]:
ctype, encoding = mimetypes.guess_type('test_html.html')
if ctype is None or encoding is not None:
    ctype = 'application/octet-stream'

maintype, subtype = ctype.split("/", 1)

In [200]:
subtype

'html'

In [70]:
final_msg = part.get_payload(decode=1).decode().replace('\n','').replace(' ', '').split('\r')

In [72]:
for word in final_msg:
    try:
        num = int(word)
        if len(word) == 6:
            print(num)
    except:
        pass

<!DOCTYPEhtml>
<html>

<head>
<title></title>
<linkrel="stylesheet"type="text/css"href="css/app.css">
<metaname="viewport"content="width=device-width,initial-scale=1.0,maximum-scale=1.0,"/>
<metaname="author"content="">
</head>

<bodystyle="margin:0px0px0px0px;padding:0px0px0px0px;">
<!--TODO:DON'TREMOVETHISCOMMENT-Injectionpointfortheinline<style>element-->
<!--<style>-->
<tablestyle="width:100%;border-spacing:0px;border:none;padding:0px0px0px;margin:0px0px0px0px;font-size:16px;line-height:17px;color:#333333;">
<tbody>
<tr>
<tdstyle="font-size:16px;line-height:17px;color:#333333;margin:0px0px0px;padding:0px0px0px;">
<tablestyle="border-spacing:0px;border:none;padding:0px0px0px;margin:0px0px0px0px;width:100%;font-family:AvenirNextLTPro,Avenir,Arial,Helvetica,sans-serif;"align="center"cellspacing="0">
<header>
<trstyle="padding:0px0px0px0px;height:60px;width:100%;">
<tdstyle="padding:6px0px6px18px;border-bottom:1pxsolid#bfbfbf;">
<imgsrc="https://accounts.intuit.com/images/v2/image_logo